In [1]:
from benedict import benedict
import yaml

import distributions


In [2]:
input_yaml = """

LIGHT_PROFILE:
    NAME: SERSIC_ELLIPSE
    PARAMETERS:
        center_x:
            DISTRIBUTION:
                NAME: uniform
                PARAMETERS:
                    minimum: -0.5
                    maximum: 0.5
        center_y:
            DISTRIBUTION:
                NAME: uniform
                PARAMETERS:
                    minimum: -0.5
                    maximum: 0.5

        R_sersic:
            DISTRIBUTION:
                NAME: normal
                PARAMETERS:
                    mean: 100
                    std: 5

        n_sersic:
            DISTRIBUTION:
                NAME: normal
                PARAMETERS:
                    mean: 100
                    std: 5

"""
with open('sersic_light_defaults.yaml', 'w+') as fake_default_file:
    fake_default_file.write(input_yaml)

In [3]:
fake_config = """
DATASET:
    NAME: DeepLenstronomyData
    PARAMETERS:
        SIZE: 100000
        OUTDIR: temp
        IMAGE_SIZE: 65

SURVEY:
    NAME: DES
    PARAMETERS:
        PIXEL_SIZE: 0.27
        SKY_BRIGHTNESS:
            DISTRIBUTION:
                NAME: normal
                PARAMTERS:
                    mean: 30.0
                    std: 1.0

SPECIES:
    GALAXY_1:
        NAME: LENS
        LIGHT_PROFILE:
            NAME: SERSIC_ELLIPSE
            PARAMETERS:
                center_x:
                    DISTRIBUTION:
                        NAME: uniform
                        PARAMETERS:
                            minimum: -0.5
                            maximum: 0.5
                center_y:
                    DISTRIBUTION:
                        NAME: uniform
                        PARAMETERS:
                            minimum: -0.5
                            maximum: 0.5

                R_sersic:
                    DISTRIBUTION:
                        NAME: normal
                        PARAMETERS:
                            mean: 100
                            std: 5

                n_sersic:
                    DISTRIBUTION:
                        NAME: normal
                        PARAMETERS:
                            mean: 100
                            std: 5
                            
        MASS_PROFILE:
            NAME: 
            PARAMETERS:
            
    GALAXY_2:
        NAME: SOURCE
        INPUT: sersic_light_defaults.yaml
        
    POINTSOURCE_1:
        NAME: AGN
        HOST: SOURCE
        PARAMETERS:
            
    POINTSOURCE_2:
        NAME: SUPERNOVA
        HOST: LENS
        PARAMETERS:
        
    POINTSOURCE_3:
        NAME: STAR
        HOST: NONE
        PARAMETERS:
        
    NOISE_1:
        NAME: POISSION_NOISE
        PARAMETERS:
        
    NOISE_2:
        NAME: DES_NOISE
        PARAMETERS:
        
GEOMETRY:

    CONFIGURATION_1:
        NAME: GALAXY_GALAXY
        FRACTION: 0.5
        PLANE_1:
            OBJECT_1: LENS
            PARAMTERS:
                REDSHIFT:
                    DISTRIBUTION:
                        NAME: delta_function
                        PARAMETERS:
                            value: 0.5
                            
        PLANE_2:
            OBJECT_1: SOURCE
            PARAMETERS:
                REDSHIFT:
                    DISTRIBUTION:
                        NAME: uniform
                        PARAMETERS:
                            minimum: 1.0
                            maximum: 1.2
                            
    CONFIGURATION_2:
        FRACTION: 0.3
        NAME: GALAXY_AGN
        PLANE_1: 
            OBJECT_1: LENS
            PARAMTERS:
                REDSHIFT:
                    DISTRIBUTION:
                        NAME: delta_function
                        PARAMETERS:
                            value: 0.5
        PLANE_2:
            OBJECT_1: SOURCE
            PARAMETERS:
                REDSHIFT:
                    DISTRIBUTION:
                        NAME: uniform
                        PARAMETERS:
                            minimum: 1.0
                            maximum: 1.2
            OBJECT_2: AGN
            
    CONFIGURATION_3:
        NAME: DOUBLE_SOURCE_PLANE_AGN
        FRACTION: 0.2
        PLANE_1:
            OBJECT_1: LENS
            PARAMETERS: 
                REDSHIFT:
                    DISTRIBUTION:
                        NAME: delta_function
                        PARAMETERS:
                            value: 0.2
        PLANE_2:
            OBJECT_1: LENS
            PARAMETERS: 
                REDSHIFT:
                    DISTRIBUTION:
                        NAME: delta_function
                        PARAMETERS:
                            value: 0.5
        PLANE_3:
            OBJECT_1: SOURCE
            PARAMETERS:
                REDSHIFT:
                    DISTRIBUTION:
                        NAME: delta_function
                        PARAMETERS:
                            value: 1.0
            OBJECT_2: AGN

"""

with open('fake_config.yaml', 'w+') as fake_config_file:
    fake_config_file.write(fake_config)

In [22]:
class Parser():
    def __init__(self, config):
        
        # Read main configuration file
        self.full_dict = self.read(config)
        
        # If the main file points to any input files, read those too
        self.get_input_locations()
        self.include_inputs()
        
        # Check for user errors in inputs
        self.check()
        
        # Determine lenstronomy tasks
        self.make_lenstronomy_tasks()
        
        return
    
    def include_inputs(self):
        """
        Adds any input yaml files to config dict and assigns to self.
        """
        config_dict = benedict(self.full_dict.copy(), keypath_separator='.')
        
        for input_path in self.input_paths:
            input_dict = self.read(config_dict[input_path + '.INPUT'])
            for k, v in input_dict.items():
                config_dict[input_path + '.{0}'.format(k)] = v
                
        self.config_dict = benedict(config_dict, keypath_separator=None)
        return    
    
    def get_input_locations(self):
        """
        Find locations in main dictionary where input files are listed
        """
        d = benedict(self.full_dict, keypath_separator='.')
        input_locs = [x.find('INPUT') for x in d.keypaths()]
        input_paths = [y for y in [x[0:k-1] if k != -1 else '' for x, k in zip(d.keypaths(), input_locs)] if y != '']
        self.input_paths = input_paths
        return
        
    
    def read(self, config):
        """
        Reads config file into a dictionary and returns it.
        
        :param config: Name of config file.
        :return: config_dict: Dictionary containing config information.
        """
        with open(config, 'r') as config_file_obj:
            config_dict = yaml.safe_load(config_file_obj)
            
        return config_dict

    
    def check(self):
        """
        Check configurations for possible user errors.
        """
        
        #BIG TODO
        return
        
    def make_lenstronomy_tasks(self):
        """
        Based on config_dict, generate a list of lenstronomy tasks
        """
        
        #ALSO BIG TODO
        return

    

In [23]:
P = Parser('fake_config.yaml')

In [24]:
P.config_dict

{'SPECIES': {'GALAXY_1': {'NAME': 'LENS',
   'LIGHT_PROFILE': {'NAME': 'SERSIC_ELLIPSE',
    'PARAMETERS': {'center_x': {'DISTRIBUTION': {'NAME': 'uniform',
       'PARAMETERS': {'minimum': -0.5, 'maximum': 0.5}}},
     'center_y': {'DISTRIBUTION': {'NAME': 'uniform',
       'PARAMETERS': {'minimum': -0.5, 'maximum': 0.5}}},
     'R_sersic': {'DISTRIBUTION': {'NAME': 'normal',
       'PARAMETERS': {'mean': 100, 'std': 5}}},
     'n_sersic': {'DISTRIBUTION': {'NAME': 'normal',
       'PARAMETERS': {'mean': 100, 'std': 5}}}}},
   'MASS_PROFILE': {'NAME': None, 'PARAMETERS': None}},
  'GALAXY_2': {'NAME': 'SOURCE',
   'INPUT': 'sersic_light_defaults.yaml',
   'LIGHT_PROFILE': {'NAME': 'SERSIC_ELLIPSE',
    'PARAMETERS': {'center_x': {'DISTRIBUTION': {'NAME': 'uniform',
       'PARAMETERS': {'minimum': -0.5, 'maximum': 0.5}}},
     'center_y': {'DISTRIBUTION': {'NAME': 'uniform',
       'PARAMETERS': {'minimum': -0.5, 'maximum': 0.5}}},
     'R_sersic': {'DISTRIBUTION': {'NAME': 'normal',


In [6]:
def convert_to_string(distribution_dict):
    return distribution_dict['NAME'] + '(' + ', '.join(['{0}={1}'.format(k, v) for k, v in distribution_dict['PARAMETERS'].items()]) + ')'

In [7]:
distribution_dict = P.options['SPECIES']['GALAXY']['LIGHT_PROFILE']['PARAMETERS']['R_sersic']['DISTRIBUTION']

draw_command = 'distributions.{0}'.format(convert_to_string(distribution_dict))

print(draw_command)

draw = eval(draw_command)

print(draw)

distributions.normal(mean=100, std=5)
104.82214433285134


In [4]:
test_dict = {'a': 'sk', 'b': 'akghj', 'c': {'d': 'kajng'}}
#d = benedict(test_dict, key_path_separator='.')

In [5]:
#d = benedict()
d = benedict(test_dict, keypath_separator='.')
d.keypaths()

['a', 'b', 'c', 'c.d']

In [17]:
d['c.d']

'kajng'